# 4.0 Pose Sequence Analysis

`PoseSequenceAnalysis` class would be designed to analyze sequential changes in various pose aspects, such as face position, body position, and others. 

1. `get_total_spin_count()`: This method counts the number of times the face position changes from front to back or back to front, considering a change valid only if it's repeated in two consecutive instances.
2. `get_total_inversions()`: This method calculates the number of times the body position shifts from upright/horizontal to inverted, with similar logic to validate the change.

(To be continued)

In [24]:
import pandas as pd
import numpy as np

class PoseSequenceAnalysis:
    def __init__(self, data):
        self.data = data
        self.spin_count = self.get_total_spin_count()
        self.invert_count = self.get_total_inversions()

    def get_total_spin_count(self):
        spin_count = 0
        faces = self.data['pos_face'].tolist()
    
        for i in range(2, len(faces)):
            if faces[i] != faces[i - 2] and faces[i] == faces[i - 1]:
                spin_count += 1

        return spin_count
        
    def get_total_inversions(self):
        inversion_count = 0
        bodies = self.data['pos_body'].tolist()
        
        for i in range(3, len(bodies)):
            if bodies[i] == 'inverted' and bodies[i - 1] != 'inverted' and bodies[i + 1] != 'inverted':
                inversion_count += 1
        return inversion_count
        

In [25]:
data = pd.read_csv('data/ground_truth/processed/upright/pose_data.csv')
results = PoseSequenceAnalysis(data)

print("Total Spin Counts:", results.spin_count)
print("Total Invert Counts:", results.invert_count)

Total Spin Counts: 5
Total Invert Counts: 0


In [26]:
data = pd.read_csv('data/ground_truth/processed/combined/pose_data.csv')
results = PoseSequenceAnalysis(data)

print("Total Spin Counts:", results.spin_count)
print("Total Invert Counts:", results.invert_count)

Total Spin Counts: 29
Total Invert Counts: 9
